In [1]:
%matplotlib notebook
from os import getcwd
from os.path import join
from sys import path
path.insert(1, join(getcwd(), '..', '..', 'ibllib', 'python'))
import dat
from alf import alf_parts, is_alf

In [6]:
from behavior_plots import *
from load_behavior import load_behavior
#import load_behavior as load_behavior
path = r'\\zubjects.cortexlab.net\Subjects'
#path = 'G:\\'
#TODO: Fix ln 51 dat: value error on list of strings
#TODO: make_pretty decorator

Windowed performance

In [5]:
print(load_behavior.load_behavior)

<function load_behavior at 0x000000000AF0ABF8>


## Loading trials and plotting
Below we load all the experiments locally and make three plots

In [7]:
refs, date, seq = dat.list_exps('LEW010', rootDir=path)
# Load all data
dfs = [load_behavior(ref, rootDir=path) for ref in refs]
dfs = [df for df in dfs if df is not None]
fig = plt.figure()
plot_perf_heatmap(dfs, plt.gca())
plt.figure()
plot_RTs(dfs[-1], plt.gca())
plt.figure()
plot_psychometric(dfs[-1], plt.gca())
plt.figure()
make_pretty(plot_windowed_perf(dfs[-1], 20, plt.gca()))

NameError: name 'dat' is not defined

## Interactive RT plot

In [ ]:
print(*[12, 321])

## Loading trials for multiple subjects

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# Subject list
subjects = ['ALK081', 'LEW008', 'LEW009', 'LEW010']
# List all experiments for each subject
refs = [dat.list_exps(subject, path)[0] for subject in subjects]
# Flatten list
#flatten = lambda l: [item for sublist in l for item in sublist]
#refs = [flatten(l) for l in refs]
# Load trials from ALFs
dfs = [[alf.load_behavior(ref, path) for ref in subject] for subject in refs]
dfs = [[df for df in subjectDfs if df is not None] for subjectDfs in dfs]
refs = [[df.name for df in subjectDfs] for subjectDfs in dfs]

In [ ]:
#wheel, wm = [[load_wheel(ref, path) for ref in subject] for subject in refs]
#wheel, wm = [[w for w in subjectW if w is not None] for subjectW in dfs]


In [ ]:
#TODO: Plot in grid
fig = plt.figure(figsize=(10, 6))
plt.clf()
grid = plt.GridSpec(3, 3, wspace=.5, hspace=.9)
ax1 = plt.subplot(grid[0, 0])
ax2 = plt.subplot(grid[0, 1])
ax3 = plt.subplot(grid[0, 2])

ax4 = plt.subplot(grid[2, :])
ax5 = plt.subplot(grid[1, 1])
ax6 = plt.subplot(grid[1, 2])
ax7 = plt.subplot(grid[1, 0])

@interact(Subject=subjects)
def subject_plots(Subject):
    plot_perf_heatmap(dfs[subjects.index(Subject)], ax1)
    ax2.cla()
    plot_learning(dfs[subjects.index(Subject)], ax2)
    plot_repeats(dfs[subjects.index(Subject)], ax=ax3)
    ax3.legend().set_visible(False)
    
    @interact(Subject=fixed(Subject), Session=(1,len(dfs[subjects.index(Subject)])))
    def session_plots(Subject, Session):
        df = dfs[subjects.index(Subject)][Session-1]
        plot_RTs(df,ax4)
        #TODO: return axes for setting title
        plot_psychometric(df,ax5)
        plot_windowed_perf(df, 20, ax6)
        ax4.legend().set_visible(False)
        fig.suptitle(df.name)

        @interact(update=False)
        def update_wheel_plot(update=True):
            if update == True:
                wheel, wm = load_wheel(df.name, rootDir=path)
                if wm is None:
                    return
                ret, = plot_wheel_at_move_onset((wheel, wm),ax7)
                ax7.set_xlim([0, 1000])
                for option in ret.keys():
                    if option in ['flinch', 'other']:
                        plt.setp(ret[option], visible=False)
    return Subject


## Performance over the session

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_learning(dfs, ax=None):
    
    nn = np.array([sum((df['contrast']>=.5) & 
                       (df['included']==True)) 
                   for df in dfs])
    pp = np.array([sum((df['contrast']>=.5) & 
                       (df['feedbackType']==1) & 
                       (df['included']==True))
                   for df in dfs]) / nn
    ci = 1.96*np.sqrt(pp*(1-pp)/nn)
    # graphics
    if ax is None:
        plt.figure()
        ax = plt.gca()
    ax.errorbar(np.arange(1,len(dfs)+1), pp, yerr=ci, capsize=2)
    ax.plot([1, len(dfs)+1], [.5, .5], 'k:')
    # Hide the right and top spines
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    # Reduce the clutter
    ax.set_xticks([1] + [i * 5 for i in range(1,round(len(dfs)/5))])
    ax.set_yticks([0, .25, .5, .75, 1.])
    # Set bounds of axes lines
    ax.spines['left'].set_bounds(.4, 1.)
    ax.spines['bottom'].set_bounds(1, len(dfs)+1)
    # Explode out axes
    #ax.spines['left'].set_position(('outward',10))
    ax.spines['bottom'].set_position(('outward',10))
    # Set the limits
    ax.set_xlim([0, len(dfs)+1])
    ax.set_ylim([.4, 1.])
    plt.xlabel('Session #')
    plt.ylabel('Performance at contrast >= 50%')
    return ax

plt.figure()
ax = plt.gca()
for i in dfs:
    plot_learning(i, ax)
plt.show()

## Do Mice adopt win-stay lose-switch?

In [ ]:
pro

plot_repeats(dfs[1], normalize=False);

In [ ]:
sum(dfs[0][0]['feedbackType'].where(dfs[0][0]['repNum']>5)==1)
print(list(range(1,6)))

## Loading the wheel files

In [ ]:
from os import listdir, getcwd
from os.path import isfile, join
from alf import *
def load_wheel(ref, rootDir=None):
    """
    Load the wheel object for a given experiment reference
    
    Example:
        df = load_wheel('2018-09-11_1_MOUSE', rootDir = r'\\server1\Subjects')
        df.head()
        
    Args: 
        subject (str): The subject name
        rootDir (str): The root directory, i.e. where the subject data are stored.
                       If rootDir is None, the current working directory is used.
        
    Returns:
        wheel (DataFrame): DataFrame constructed from the wheel object of the ALF 
                           files located in the experiment directory
        wm (DataFrame): DataFrame constructed from the wheelMoves object of the  
                           ALF files located in the experiment directory

    TODO: Deal with namespaces: currently hard-coded
    TODO: Make function more efficient: Does everything twice (once per ALF obj)
    TODO: Extract first few lines as decorator
    """
    if rootDir is None:
        rootDir = getcwd()
    path = dat.exp_path(ref, rootDir)
    alfs = [f for f in listdir(path) if (isfile(join(path, f))) & (is_alf(f)) & (f.startswith('_ibl_wheel'))]
    if not alfs:
        print('{}: Nothing to process'.format(ref))
        return None, None
    # Pull paths of trials ALFs
    wheelPos = np.load(join(path, '_ibl_wheel.position.npy')).squeeze()
    wheelVel = np.load(join(path, '_ibl_wheel.velocity.npy')).squeeze()
    t = np.load(join(path, '_ibl_wheel.timestamps.npy')).squeeze()
    times = np.interp(np.arange(0,len(wheelPos)), t[:,0], t[:,1])
    wheel = pd.DataFrame({'position':wheelPos, 'velocity':wheelVel, 'times':times})
    
    intervals = np.load(join(path, '_ibl_wheelMoves.intervals.npy')).squeeze()
    try:
        movesType = pd.read_csv(join(path, '_ibl_wheelMoves.type.csv'), header=None)
        wm = pd.DataFrame({'onset':intervals[:,0], 'offset':intervals[:,1], 'type':movesType.values[0]})
    except: #TODO: Deal with missing movesType or empty file
        wm = None

    return wheel, wm

## Looking at wheel moves

In [ ]:
wheel, wm = load_wheel(refs[1][-2], rootDir=path)
#def plot_wheel_at_move_onset()
f = plt.figure()
ax = f.gca()
colours = {'CW':'b', 'CCW':'r', 'flinch':'k', 'other':'k'}
ret = {'CW':[], 'CCW':[], 'flinch':[], 'other':[]}
for i in range(0,len(wm)):
    t = (wheel['times'] > wm['onset'][i]) & (wheel['times'] < wm['offset'][i])
    pos = wheel['position'][t]
    wheelTimes = wheel['times'][t]
    relativeTimes = wheelTimes - wheelTimes.iloc[0]
    pos = pos - pos.iloc[0]
    ln, = ax.plot(relativeTimes*1000, pos.values, c=colours[wm['type'][i]], label=wm['type'][i])
    ret[wm['type'][i]].append(ln)
    
plt.xlim([0, 10000])
plt.ylim([-7, 7])
# Hide the right and top spines
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
# Set bounds of axes lines
#ax.spines['left'].set_bounds(0, 1)
#ax.spines['bottom'].set_bounds(1, len(dfs)+1)
# Explode out axes
ax.spines['left'].set_position(('outward',10))
ax.spines['bottom'].set_position(('outward',10))
plt.title = refs[1][-2];
plt.xlabel('Time from movement onset (ms)')
plt.ylabel('Relative position (cm)')


plt.show()

btn = widgets.SelectMultiple(options=ret.keys(), value=['CCW', 'CW'], description='Show:')
slider = widgets.IntRangeSlider(
    value=[0, 1000],
    min=0,
    max=10000,
    step=1,
    description='x limit:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)
@interact(Selected=btn, lim=slider)
def set_visible(Selected, lim):
    for option in ret.keys():
        if option in Selected:
            plt.setp(ret[option], visible=True)
            #ax.plot([0, 500], [-6, 4], 'r-')
            #[ln.set_visible(True) for ln in ret[option]]
        else:
            plt.setp(ret[option], visible=False)
    ax.set_xlim(lim)
    plt.draw()

In [ ]:
def plot_wheel_at_move_onset(wheelData, ax=None):
    if ax is None:
        plt.figure()
        ax = plt.gca()
    ax.cla()
    wheel = wheelData[0]
    wm = wheelData[1]
    colours = {'CW':'b', 'CCW':'r', 'flinch':'k', 'other':'k'}
    ret = {'CW':[], 'CCW':[], 'flinch':[], 'other':[]}
    for i in range(0,len(wm)):
        t = (wheel['times'] > wm['onset'][i]) & (wheel['times'] < wm['offset'][i])
        pos = wheel['position'][t]
        wheelTimes = wheel['times'][t]
        relativeTimes = wheelTimes - wheelTimes.iloc[0]
        pos = pos - pos.iloc[0]
        ln, = ax.plot(relativeTimes*1000, pos.values, c=colours[wm['type'][i]], label=wm['type'][i])
        ret[wm['type'][i]].append(ln)
    ax.set_xlim([0, 10000])
    ax.set_ylim([-7, 7])
    # Hide the right and top spines
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    # Set bounds of axes lines
    #ax.spines['left'].set_bounds(0, 1)
    #ax.spines['bottom'].set_bounds(1, len(dfs)+1)
    # Explode out axes
    ax.spines['left'].set_position(('outward',10))
    ax.spines['bottom'].set_position(('outward',10))
    ax.set_xlabel('Time from movement onset (ms)')
    ax.set_ylabel('Relative position (cm)')
    return ret

## Moves at stim on

In [ ]:
wheel, wm = load_wheel(refs[1][-2], rootDir=path)
trials = alf.load_behavior(refs[1][-2], rootDir=path)
print(list(trials.columns.values))
plt.figure()
ax = plt.gca()
for i in range(0,len(trials)):
    t = (wheel['times'] > trials['stimOn_times'][i]-1) & (wheel['times'] < trials['response_times'][i])
    pos = wheel['position'][t]
    wheelTimes = wheel['times'][t]
    relativeTimes = wheelTimes - wheelTimes.iloc[0]
    pos = pos - pos.iloc[0]
    if len(pos) > 1000*60:
        print(trials['stimOn_times'][i])
        print(trials['response_times'][i])
        continue
    #if trials['choice'][i] == 1:
    #    c = 'g-'
    #elif trials['choice'][i] == -1:
    #    c = 'r-'
    #else:
    #    c = 'k-'
    if trials['feedbackType'][i] == 1:
        c = 'g-'
    elif trials['feedbackType'][i] == -1:
        c = 'r-'
    else:
        c = 'k-'

    ax.plot(relativeTimes*1000, pos, c)
    ax.set_xlim([-10, 1000])
plt.show()

In [ ]:
trials.head()

## Determine if a mouse has learned the task

In [ ]:
import pandas as pd
def is_learned(dfs, verbose=False, returnIndex=False):
    """
    Determine whether the mouse has met the criteria for having learned
    
    Example:
        
    Args: 
        dfs (list): List of data frames constructed from an ALF trials object.
        verbose (bool): If True, prints the list of conditions that were(n't) met.
        returnIndex: If True, returns the index of session on which mouse first 
                    met the 'learned' criteria.  This can take longer to process.
        
    Returns:
        learned (bool or int): if returnIndex is True, returns a bool indicating 
                if the mouse has met the criteria, otherwise returns the index 
                of the session on which the mouse was first met those criteria.
        
    TODO: Should this take a mouse name as input instead?
    TODO: Create conditions list, print list at end of function
    """
    criteria = ['asymmetric trials already introduced',
               'full contrast set introduced',
               'over 300 trials on three consecutive sessions',
               'performance at high contrast over 80% on three consecutive sessions'
               'absolute bias below 16',
               'threshold below 19',
               'lapse rate below 20%']
    learned = False
    j = 0
    for i in range(0,len(dfs),-1):
        # If trial side prob uneven, the subject must have learned
        if any(dfs[i]['probabilityLeft']!=0.5):
            if not returnIndex:
                learned = True
                if verbose == True:
                    print('Asymmetric trials already introduced')
                break
        # If there are fewer than 4 contrasts, subject can't have learned
        elif len(dfs[i]['contrast'].unique()) < 4:
            if verbose == True:
                print('Low contrasts not yet introduced')
            if returnIndex:
                learned = None
            break
        else:
            perfOnEasy = (sum(dfs[i]['feedbackType']==1. & abs(dfs[i]['contrast']) > .25)/
                          sum(abs(dfs[i]['contrast'])))
            if len(dfs[i]) > 200 & perfOnEasy > .8:
                if j < 2:
                    j += 1
                else: # All three sessions meet criteria
                    df = pd.concat(dfs[i:i+3])
                    contrastSet = np.sort(df['contrast'].unique())
                    nn = np.array([sum((df['contrast']==c) & (df['included']==True)) for c in contrastSet])
                    pp = np.array([sum((df['contrast']==c) & (df['included']==True) & (df['choice']==-1.)) for c in contrastSet])/nn
                    pars, L = psy.mle_fit_psycho(np.vstack((contrastSet,nn,pp)), 
                                     P_model='erf_psycho',
                                     parstart=np.array([np.mean(contrastSet), 3., 0.05]),
                                     parmin=np.array([np.min(contrastSet), 10., 0.]), 
                                     parmax=np.array([np.max(contrastSet), 30., .4]))
                    if abs(pars[0]) > 16:
                        if verbose == True:
                            print('Absolute bias too high')
                            break
                    if pars[1] > 19:
                        if verbose == True:
                            print('Threshold too high')
                            break
                    if pars[2] > .2:
                        if verbose == True:
                            print('Lapse rate too high')
                            break
                    if verbose == True:
                        print('Mouse learned')
                    learned = True
            else:
                if verbose == True:
                    print('Low trial count or performance at high contrast')
                break
                
    if returnIndex & (not learned):
        return None
    elif returnIndex & learned:
        return i + 3
    else:
        return learned

In [ ]:
print(is_learned(dfs, verbose=True, returnIndex=False))

## Looking at trial side manipulations

In [ ]:
#TODO: Change plot_psychometric to split by side prob
def plot_choice_by_side(df, ax=None):
    if ax is None:
        plt.figure()
        ax = plt.gca()
    ax.scatter(df['contrast'][df['choice']==1], 
                df.index.values[df['choice']==1]+1, 
                s=100, marker='_', c='r')
    ax.scatter(df['contrast'][df['choice']==-1], 
                df.index.values[df['choice']==-1]+1, 
                s=100, marker='_', c='b')
def plot_choice_windowed(df, window=10, ax=None):
    if ax is None:
        plt.figure()
        ax = plt.gca()
    # May require raw=False arg in older versions
    pctRight = df['choice'].rolling(window).apply(lambda x: sum(x==1)/len(x))
    ax.plot(pctRight, df.index.values+1)
    ax.plot((.5,.5), (1,len(df)), 'k--')
    ax.set_xlim([0,1.])
    
plot_choice_windowed(dfs[0][1])

## Getting weight information

In [ ]:
from webclient import AlyxClient
import datetime
#baseURL = https://alyx.internationalbrainlab.org/
baseURL = 'https://alyx.cortexlab.net/'
ai = AlyxClient(username='miles', password=pwd, base_url=baseURL)

In [ ]:
def get_weight_records(subjects, ai):
    s = ai.get('subjects?stock=False')
    rmKeys = ['actions_sessions','water_administrations','weighings','genotype']
    subject_info = []
    records = []
    weight_info = []
    for s in subjects:
        subj = ai.get('subjects/{}'.format(s))
        subject_info.append({key: subj[key] for key in subj if key not in rmKeys})
        endpoint = ('water-requirement/{}?start_date=2016-01-01&end_date={}'
                    .format(s, datetime.datetime.now().strftime('%Y-%m-%d')))
        wr = ai.get(endpoint)
        if wr['implant_weight']:
            iw = wr['implant_weight']
        else:
            iw = 0
        #TODO MultiIndex without None
        if not wr['records']:
            records.append(None)
        else:
            df = pd.DataFrame(wr['records'])
            df = (df.set_index(pd.DatetimeIndex(df['date']))
                  .drop('date', axis=1)
                  .assign(pct_weight = lambda x: 
                          (x['weight_measured']-iw) /
                          (x['weight_expected']-iw) 
                          if 'weight_measured' in x.columns.values 
                          else np.nan))
            records.append(df)
            wr.pop('records', None)
        weight_info.append(wr)

    
    info = (pd.DataFrame(weight_info)
            .merge(pd.DataFrame(subject_info), left_on='subject', right_on='nickname')
           .set_index('subject'))
    records = pd.concat(records, keys=subjects, names=['name', 'date'])
    return records, info
records, info = get_weight_records(subjects, ai)
records.head()

In [ ]:
def plot_session_pct_weight(dfs, ax=None):
    #TODO: Index lookup rather than rely on order
    if ax is None:
        plt.figure()
        ax = plt.gca()
    idx = pd.IndexSlice
    weightPCT = []
    numTrials = []
    for i,s in enumerate(records.index.get_level_values('name').unique()): 
        df = dfs[i]
        expRef = [block.name for block in df]
        date = [dat.parse_ref(ref)[1] for ref in expRef]
        numTrials.append([len(block) for block in df if dat.parse_ref(df.name)[1] is in records.loc[idx[s, :]])
        weightPCT.append(records.loc[idx[s, :], 'pct_weight'].values)
    #ax.plot(np.array(numTrials),np.array(weightPCT))
    return np.array(numTrials),np.array(weightPCT)
n, w = plot_session_pct_weight(dfs)
print(w)

In [ ]:
s='ALK081'
print(records.index.get_level_values('name').unique())

## Getting weight information for all mice
TODO: Package into function

In [ ]:
from webclient import AlyxClient
import datetime
#baseURL = https://alyx.internationalbrainlab.org/
baseURL = 'https://alyx.cortexlab.net/'
ai = AlyxClient(username='miles', password=pwd, base_url=baseURL)

s = ai.get('subjects?stock=False')
rmKeys = ['actions_sessions','water_administrations','weighings','genotype']
subject_info = [{key: subj[key] for key in subj if key not in rmKeys} for subj in s]
subject_info = pd.DataFrame(subject_info).set_index('nickname')
records = []
weight_info = []
for s in subject_info.index.values:
    endpoint = ('water-requirement/{}?start_date=2016-01-01&end_date={}'
                .format(s, datetime.datetime.now().strftime('%Y-%m-%d')))
    wr = ai.get(endpoint)
    if wr['implant_weight']:
        iw = wr['implant_weight']
    else:
        iw = 0
    #TODO MultiIndex without None
    if not wr['records']:
        records.append(None)
    else:
        df = pd.DataFrame(wr['records'])
        df = (df.set_index(pd.DatetimeIndex(df['date']))
              .drop('date', axis=1)
              .assign(pct_weight = lambda x: 
                      (x['weight_measured']-iw) /
                      (x['weight_expected']-iw) 
                      if 'weight_measured' in x.columns.values 
                      else np.nan))
        records.append(df)
        wr.pop('records', None)
    weight_info.append(wr)

info = (pd.DataFrame(weight_info)
        .merge(subject_info, left_on='subject', right_index=True)
       .set_index('subject'))
records = pd.concat(records, keys=subject_info.index.values, names=['name', 'date'])

records.head()

In [ ]:
info.to_pickle('info.pkl')
records.to_pickle('records.pkl')

In [ ]:
info = pd.read_pickle('info.pkl')
records = pd.read_pickle('records.pkl')

## Plotting meta-data

In [ ]:
def plot_weight(records, ax=None):
    if ax is None:
        plt.figure()
        ax = plt.gca()
    idx = pd.IndexSlice
    for s in records.index.get_level_values('name'): 
        gt80 = ((records.loc[idx[s, :], 'pct_weight'] > 0.8) | 
                (records.loc[idx[s, :], 'pct_weight'].isnull()))
        gt70 = ((records.loc[idx[s, :], 'pct_weight'] < 0.8) & 
                (records.loc[idx[s, :], 'pct_weight'] > 0.7) | 
                (records.loc[idx[s, :], 'pct_weight'].isnull()))
        lt70 = ((records.loc[idx[s, :], 'pct_weight'] < 0.7) | 
                (records.loc[idx[s, :], 'pct_weight'].isnull()))
        wtrDays = records.loc[idx[s, :], :].index.get_level_values(1)
        wtrDays = wtrDays-wtrDays[0]
        ax.plot(wtrDays[gt80], records.loc[idx[s, :], 'pct_weight'][gt80], 'k-')
        ax.plot(wtrDays[gt70], records.loc[idx[s, :], 'pct_weight'][gt70], 'y-')
        ax.plot(wtrDays[lt70], records.loc[idx[s, :], 'pct_weight'][lt70], 'r-')
    return ax
plt.figure()
ax = plt.gca()
plot_weight(records, ax)
ax.set_ylim([0.7,1.25])
ax.set_xlim([0,25e14])

In [ ]:
from matplotlib.offsetbox import AnchoredText
def mean_weight_change(records):
    idx = pd.IndexSlice
    for s in records.index.get_level_values('name'):    
        gt70 = (records.loc[idx[s, :], 'pct_weight'] > 0.7)
        wtrDays = records.loc[idx[s, :], :].index.get_level_values(1)
        records.loc[idx[s, :], 'days_from_start'] = wtrDays - wtrDays[0]
        #ls.append(records.loc[idx[s, :], 'pct_weight'][gt70].values)
mean_weight_change(records)

In [ ]:
records = (records.reset_index()
           .set_index(['name', 'days_from_start'])
           .replace([np.inf, -np.inf], np.nan))
records['pct_weight'] = records['pct_weight'].where(records['pct_weight']>0.6)
r = records.groupby(level=1,axis=0)['pct_weight'].describe()[['mean','std']]
r['std'] = r['std'].where(r['std']<.5)
r.plot(yerr='std', fmt='k-o')
#plt.figure()
ax = plt.gca()
#ax.errorbar(r.index.days, r['mean'], yerr=r['std'])
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
# Set bounds of axes lines
ax.spines['left'].set_bounds(.8, 1.1)
ax.spines['bottom'].set_bounds(0, 14)
# Explode out axes
ax.spines['left'].set_position(('outward',10))
ax.spines['bottom'].set_position(('outward',10))
# Specify tick label size
ax.tick_params(axis = 'y', which = 'major')
ax.tick_params(axis = 'y', which = 'minor', labelsize = 0)
# Suppress minor tick labels
ax.set_yticks(np.arange(.8,1.11,.1))
ax.set_yticks(np.arange(.8,1.11,.05), minor = True)
ax.set_xticks(np.arange(0,101,7))
ax.set_xticklabels(np.arange(0,101,7))
#ax.set_xticks(np.arange(.8,1.11,.05), minor = True)
ax.set_xlim([-.2,21])
ax.set_ylim([.8,1.1])
ax.legend().set_visible(False)
ax.set_title('Mean percentage weight from first water restriction')
anchored_text = AnchoredText('n = 90', loc=1, frameon=False)
ax.add_artist(anchored_text)
plt.xlabel('Days since water restriction')
plt.ylabel('z-scored weight percent of initial weight')

slider = widgets.IntRangeSlider(
    value=(-1, 21),
    min=-1,
    max=100,
    step=1,
    description='x limit:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)
@interact(lim=slider)
def set_xlim(lim):
    lim = list(lim)
    if lim[0]==-1:
        lim[0] = -.2
    ax.set_xlim(lim)
    #an.annotate('n = 190', xy=(18, 1.05), xytext=(lim[1], 1.05))
    ax.spines['bottom'].set_bounds(*lim)

In [ ]:
#idx = pd.IndexSlice
#records = (records.reset_index()
#           .set_index(['name', 'date'])
#           .replace([np.inf, -np.inf], np.nan))
plt.figure()
ax = plt.gca()
for s in records.index.get_level_values('name'): 
    #ax.plot(records.sort_index(level=0).loc[idx[s, '2018-06-08':'2018-09-01'], 'weight_measured'])
    rec = records.sort_index(level=0).loc[s,:,:].reset_index(['name'])
    ax.plot(records.loc['2018-06-08':'2018-09-01', 'weight_measured'])
    print(rec)
    #ax.plot(records.loc[(s, '2018-06-08'):(s, '2018-09-01'),'weight_measured'])
    
ax.plot([datetime('2018-06-09'), datetime('2018-06-09')], [0,100])

In [ ]:
records.head()

In [ ]:
import scipy.io
mat = scipy.io.loadmat('\\zserver.cortexlab.net\Lab\Share\Miles\behaviour\subjectData.mat')

In [ ]:
def is_alf(fileName):
    """
    Returns a True for a given file name if it is an ALF file, otherwise 
    returns False
    
    Examples:
        match = is_alf('trials.feedbackType.npy')
        match == True
        >> True
        match = is_alf('config.txt')
        match == False
        >> True
    
    Args:
        fileName (str): The name of the file
        
    Returns:
        bool
    
    @author: Miles
    """
    pattern = r'(?P<nsp>_.+_)?(?P<obj>.+)\.(?P<typ>.+)\.(?P<ext>.+)'
    out = re.match(pattern, fileName)
    return out is not None
def alf_parts(fileName):
    """
    Return the object, type and extention for a given ALF file name
    
    Examples:
        obj, typ, ext = alf_parts('trials.choice.npy')
        (None, 'trials', 'choice', 'npy')
        obj, typ, ext = alf_parts('_misc_trials.choice.npy')
        ('_misc_', 'trials', 'choice', 'npy')
    
    Args:
        fileName (str): The name of the file
        
    Returns:
        nsp (str): The namespace, if present
        obj (str): ALF object
        typ (str): The ALF attribute
        ext (str): The file extension
        
    TODO: Deal with namespaces
    
    @author: Miles
    """
    try:
        pattern = r'(?P<nsp>_.+_)?(?P<obj>.+)\.(?P<typ>.+)\.(?P<ext>.+)'
        out = re.match(pattern, fileName)
    except:
        print(fileName)
    return out.groupdict()
alf_parts('trials.feedbackType.npy')